In [ ]:
!pip install transformers datasets torchaudio torchcodec jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 65.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, load_from_disk, Audio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model, Wav2Vec2Config, Wav2Vec2ForPreTraining
import torch
from torch.optim import AdamW
import jiwer
from torch.utils.data import DataLoader
import gc
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


------------------------------------------------
#Pretrained Model
------------------------------------------------

In [ ]:
wav2vec_config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base-960h")

# saving config to JSON file
config_dict = wav2vec_config.to_dict()
with open(f"wav2vec2_config.json", "w") as F:
	json.dump(config_dict, F, indent=4)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

In [ ]:
# reading config from JSON file
with open(f"wav2vec2_config.json", "r") as F:
	config_dict = json.load(F)

# creating an instance of Wav2Vec2Config
wav2vec_config = Wav2Vec2Config(**config_dict)

In [ ]:
pretrained_model = Wav2Vec2ForPreTraining(wav2vec_config)

In [ ]:
pretrained_model

Wav2Vec2ForPreTraining(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (

---------------------------------------------------------
#*********************************************************
---------------------------------------------------------

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

model.to(device)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

-------------------------------------
# Pretrained model with lm_head
_______________________________________

In [ ]:
state_dict_pretrained_model = pretrained_model.state_dict()

In [ ]:
missing, unexpected = model.load_state_dict(state_dict_pretrained_model, strict=False)

In [ ]:
print("Missing keys in model:", missing)

Missing keys in model: ['lm_head.weight', 'lm_head.bias']


In [ ]:
print("Unexpected keys from pretrained model:", unexpected)

Unexpected keys from pretrained model: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_hid.bias', 'project_q.weight', 'project_q.bias']


In [ ]:
model.lm_head.reset_parameters()

---------------------------------
#********************************************
---------------------------------

In [ ]:
# dataset_10min = load_dataset(
#     "parquet",
#     data_files="/content/drive/MyDrive/SP/SP/0000.parquet"
# )
dataset_10min = load_from_disk("/content/drive/MyDrive/SP/SP/librispeech_datasets/dataset_10h")
dataset_val = load_from_disk("/content/drive/MyDrive/SP/SP/librispeech_datasets/dataset_val_clean")
# dataset_test = load_from_disk("/content/drive/MyDrive/SP/SP/librispeech_datasets/dataset_test_clean")

In [ ]:
dataset_10min

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id', 'duration'],
    num_rows: 2850
})

In [ ]:
dataset_10min[100]

{'file': '/home/albert/.cache/huggingface/datasets/downloads/extracted/bc0d9a6ef85c2d487c9c6efbc91f8892df927c69d3f80545a668cc058d5f677e/89-219-0061.flac',
 'audio': <datasets.features._torchcodec.AudioDecoder at 0x7ad4c413fd40>,
 'text': 'IN SPITE OF ALL HIS ATTENTIONS HE HAD PREVIOUSLY EXCITED AND WHAT HAD BEEN TERROR AND DISLIKE BEFORE WAS NOW ABSOLUTE AVERSION YES AVERSION HIS CRUELTY TO SUCH A CHARMING WOMAN MADE HIM ODIOUS TO HER',
 'speaker_id': 89,
 'chapter_id': 219,
 'id': '89-219-0061',
 'duration': 13.595}

In [ ]:
dataset_val[100]

{'file': '/home/albert/.cache/huggingface/datasets/downloads/extracted/234bf103726646f6948bab918be3a226316832f62b2ca6861100cff043922b4a/2035-147960-0005.flac',
 'audio': <datasets.features._torchcodec.AudioDecoder at 0x7ad4c01e9910>,
 'text': 'I WHIRLED ROUND AND THERE ON ONE OF THOSE DRY GRAVEL BEDS WAS THE BIGGEST SNAKE I HAD EVER SEEN',
 'speaker_id': 2035,
 'chapter_id': 147960,
 'id': '2035-147960-0005'}

In [ ]:
# dataset_10min = dataset_10min['train'].select(range(50))

In [ ]:
def evaluate(data_loader):
  all_refs = []
  all_preds = []

  for batch in tqdm(data_loader):
      input_values = batch["input_values"].to(device)
      attention_mask = batch["attention_mask"].to(device)

      with torch.no_grad():
          logits = model(input_values, attention_mask=attention_mask).logits

      pred_ids = torch.argmax(logits, dim=-1)
      preds = processor.batch_decode(pred_ids)

      labels_ids = batch["labels"].clone()
      labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
      refs = processor.batch_decode(labels_ids, group_tokens=False)

      # collect as strings, not token IDs
      all_preds.extend(preds)
      all_refs.extend(refs)

  import jiwer
  wer = jiwer.wer(all_refs, all_preds)
  print("Total Validation WER:", wer)


In [ ]:
import torch
from torch.utils.data import DataLoader
import random

def CreateDataLoader(dataset, batch_size=1, shuffle=False, processor=processor,
                     mask_time_prob=0.00, mask_time_length=10,
                     mask_channel_prob=0.000, mask_channel_span=64):

    def prepare_dataset(batch):
        audio = batch["audio"]
        batch["input_values"] = processor(
            audio["array"],
            sampling_rate=audio["sampling_rate"]
        ).input_values[0]

        batch["labels"] = processor.tokenizer(batch["text"]).input_ids
        return batch

    def apply_spec_augment(input_values, attention_mask):
        B, T = input_values.shape

        # ---- Time masking ----
        num_masked_spans = int(mask_time_prob * T / mask_time_length)
        for _ in range(num_masked_spans):
            start = random.randint(0, max(0, T - mask_time_length))
            input_values[:, start:start+mask_time_length] = 0.0
            attention_mask[:, start:start+mask_time_length] = 0

        # ---- Channel masking ----
        C = input_values.shape[1] if len(input_values.shape) > 1 else 1
        num_masked_channels = int(mask_channel_prob * C / mask_channel_span)
        for _ in range(num_masked_channels):
            start = random.randint(0, max(0, C - mask_channel_span))
            if C > 1:
                input_values[:, start:start+mask_channel_span] = 0.0

        return input_values, attention_mask

    def collate_fn(batch):
        input_values = [torch.tensor(f["input_values"], dtype=torch.float) for f in batch]
        labels = [torch.tensor(f["labels"], dtype=torch.long) for f in batch]

        # Pad inputs
        input_values = torch.nn.utils.rnn.pad_sequence(
            input_values, batch_first=True, padding_value=0.0
        )

        # Pad labels and replace padding with -100
        labels = torch.nn.utils.rnn.pad_sequence(
            labels, batch_first=True, padding_value=processor.tokenizer.pad_token_id
        )
        labels[labels == processor.tokenizer.pad_token_id] = -100

        # Attention mask
        attention_mask = (input_values != 0.0).long()

        # Apply SpecAugment masking
        input_values, attention_mask = apply_spec_augment(input_values, attention_mask)

        return {"input_values": input_values, "attention_mask": attention_mask, "labels": labels}

    dataset = dataset.map(prepare_dataset)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)

    return data_loader



In [ ]:
train_loader = CreateDataLoader(dataset_10min, shuffle=True)
valid_loader = CreateDataLoader(dataset_val, shuffle=False)
test_loader = CreateDataLoader(dataset_test, shuffle=False)

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-6)

# # Freeze the feature encoder completely
# for param in model.wav2vec2.feature_extractor.parameters():
#     param.requires_grad = True

In [ ]:
# total_updates = 1500*8
# warmup_ratio = 0.1
# constant_ratio = 0.4
# decay_ratio = 0.5

# warmup_steps = int(total_updates * warmup_ratio)
# constant_steps = int(total_updates * constant_ratio)
# decay_steps = int(total_updates * decay_ratio)

# print(warmup_steps, constant_steps, decay_steps)

# def tri_stage_lr(current_step, base_lr):
#     if current_step < warmup_steps:
#         # Linear warmup: 0 → base_lr
#         return base_lr * (current_step / warmup_steps)
#     elif current_step < warmup_steps + constant_steps:
#         # Constant stage
#         return base_lr
#     else:
#         # Linear decay to 0
#         decay_step = current_step - warmup_steps - constant_steps
#         return base_lr * max(0.0, 1 - decay_step / decay_steps)

# # Function to control which parameters are trainable
# def set_trainable(global_step, transformer_start_step=10000):
#     if global_step < transformer_start_step:
#         # Freeze transformer
#         for param in model.wav2vec2.encoder.parameters():
#             param.requires_grad = False
#         # Ensure classifier is trainable
#         for param in model.lm_head.parameters():
#             param.requires_grad = True
#     else:
#         # Unfreeze transformer
#         for param in model.wav2vec2.encoder.parameters():
#             param.requires_grad = True
#         for param in model.lm_head.parameters():
#             param.requires_grad = True

In [ ]:
from tqdm import tqdm

num_epochs = 800
global_step = 0
model.train()

for epoch in range(num_epochs):
    loop = tqdm(train_loader, leave=True)
    total_loss = 0.0

    for batch in loop:
        # Control trainable parameters based on global_step
        # set_trainable(global_step, transformer_start_step=10000)

        input_values = batch["input_values"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_values, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()

        #Update learning rate
        # lr = tri_stage_lr(global_step, base_lr=5e-6)  # choose base_lr
        # for param_group in optimizer.param_groups:
        #     param_group['lr'] = lr

        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        global_step += 1

        total_loss += loss.item()
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item())

    #     if global_step >= total_updates:
    #         break


    # if global_step >= total_updates:
    #         break

    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader)}")
    if (epoch+1) % 50 == 0:
      evaluate(valid_loader)


Epoch 1: 100%|██████████| 50/50 [00:26<00:00,  1.86it/s, loss=532]


Epoch 1 Loss: 1062.4716769409179


Epoch 2: 100%|██████████| 50/50 [00:22<00:00,  2.23it/s, loss=697]


Epoch 2 Loss: 613.9884909057618


Epoch 3: 100%|██████████| 50/50 [00:24<00:00,  2.00it/s, loss=690]


...

Epoch 453: 100%|██████████| 50/50 [00:17<00:00,  2.88it/s, loss=67.9]


Epoch 453 Loss: 89.12205078125


Epoch 454: 100%|██████████| 50/50 [00:16<00:00,  3.02it/s, loss=98.8]


Epoch 454 Loss: 91.89169189453125


Epoch 455:   4%|▍         | 2/50 [00:00<00:13,  3.53it/s, loss=39.4]